In [1]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization.converters import (InequalityToEquality, IntegerToBinary, 
                                            LinearEqualityToPenalty, LinearInequalityToPenalty, MaximizeToMinimize)
from docplex.mp.model import Model

# SciPy minimizer routine
from scipy.optimize import minimize

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session
from qiskit.result import QuasiDistribution
from qiskit.circuit.library import QAOAAnsatz

In [2]:
#service = QiskitRuntimeService()

In [41]:
class QuadraticProgram2Ising:
    """A mapping of `QuadraticProgram` to a Ising Hamiltonian
    in `SparsePauliOp` format
    """
    def __init__(self):
        self.input_types = (QuadraticProgram, )
        self.output_types = (SparsePauliOp, )

    def run(self, program, ansatz = None):
        if not isinstance(program, self.input_types):
            raise TypeError(f"Program is invalid input type, must be one of {self.input_types}")
        ising, constant = program.to_ising(opflow=False)
        return ising + SparsePauliOp.from_list([("I"*ising.num_qubits, constant)])

In [42]:
class QAOASolver:
    def __init__(self, session, qaoa_reps=2):
        self.input_types = (SparsePauliOp, )
        self.output_types = (QuasiDistribution, )
        self.session = session
        self.qaoa_reps = 2

    def run(self, hamiltonian):
        ansatz = QAOAAnsatz(hamiltonian, reps=2)
        estimator = Estimator(session=session, options={"shots": int(1e4)})
        sampler = Sampler(session=session, options={"shots": int(1e4)})

        def cost_func(params, ansatz, hamiltonian, estimator):
            """Return estimate of energy from estimator
        
            Parameters:
                params (ndarray): Array of ansatz parameters
                ansatz (QuantumCircuit): Parameterized ansatz circuit
                hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
                estimator (Estimator): Estimator primitive instance
        
            Returns:
                float: Energy estimate
            """
            cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
            return cost

        x0 = 2 * np.pi * np.random.rand(ansatz.num_parameters)
        res = minimize(cost_func, x0, args=(ansatz, hamiltonian, estimator), method="COBYLA")
        # Assign solution parameters to ansatz
        qc = ansatz.assign_parameters(res.x)
        # Add measurements to our circuit
        qc.measure_all()
        # Sample ansatz at optimal parameters
        samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
        # Close the session since we are now done with it
        session.close()
        return samp_dist

In [43]:
import copy
import warnings

class CompositeWorkflow:
    def __init__(self, passes):
        self.passes = passes
        self.input_types = None
        self.output_types = None
        self.strict_validation = True
        self._validate_passes()

    def _validate_passes(self):
        self.input_types = self.passes[0].input_types
        input_types = self.passes[0].input_types
        for individual_pass in self.passes:
            temp_input_types = individual_pass.input_types
            input_overlap = set(input_types).intersection(set(temp_input_types))
            if not input_overlap:
                raise Exception(f"{temp_input_types} not compatible with {input_types}")
            if len(input_overlap) != len(temp_input_types) and self.strict_validation:
                self.strict_validation = False
            input_types = individual_pass.output_types
        self.output_types = individual_pass.output_types
        if not self.strict_validation:
            warnings.warn('CompositeWorkflow is only weakly validated')
            
        
    
    def run(self, input):
        temp = copy.deepcopy(input)
        for individual_pass in self.passes:
            temp = individual_pass.run(temp)
        return temp

In [44]:
mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.binary_var("y")
z = mdl.binary_var("z")
mdl.minimize(x - 2*y + 3*z + x*y + x*z + 2*y*z)
#print(mdl.export_as_lp_string())

In [45]:
qp = QuadraticProgram.from_docplex_mp(mdl)

In [46]:
#backend = service.get_backend('ibmq_qasm_simulator')

In [47]:
#session = Session(backend=backend)

In [49]:
cw = CompositeWorkflow([InequalityToEquality(), # Transformation
                        IntegerToBinary(), # Transformation
                        LinearEqualityToPenalty(), # Transformation
                        LinearInequalityToPenalty(), # Transformation
                        MaximizeToMinimize(), # Transformation
                        #QuadraticProgram2Ising(), # Conversion
                        #QAOASolver(session) # Execute
                        ])

In [30]:
cw.input_types

(qiskit_optimization.problems.quadratic_program.QuadraticProgram,)

In [31]:
cw.output_types

(qiskit_optimization.problems.quadratic_program.QuadraticProgram,)

In [13]:
qubo = cw.run(qp)

In [14]:
qubo

<QuadraticProgram: minimize x*y + x*z + 2*y*z + x - 2*y + 3*z, 3 variables, 0 constraints, 'docplex model'>

In [2]:
def evaluate_quadratic_program(bitstring, program):
    constant = program.objective.constant
    linear_elements = program.objective.linear.to_dict()
    quadratic_elements = program.objective._quadratic.to_dict()

    # Flip string so 0th bit is 0th array element for easy math
    x = np.fromiter(list(bitstring[::-1]), dtype=int)
    
    sum = constant
    for element, val in linear_elements.items():
        sum += x[element] * val
    for element, val in quadratic_elements.items():
        sum += x[element[0]] * val * x[element[1]]
    return sum

In [26]:
results = []
for bitstring in probs:
    results.append(evaluate_quadratic_program(bitstring, qubo))
    
results

NameError: name 'qubo' is not defined

In [358]:
qubo.variables

[<Variable: x (binary)>, <Variable: y (binary)>, <Variable: z (binary)>]